In [1]:
import time
import re, ast
import numpy as np
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'



In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1") # "local[2]" to run locally with 2 cores
sc = SparkContext(conf=conf) 
sqlContext=SQLContext(sc)
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [3]:
ssc = StreamingContext(sc, 5) 
# 0.1 = batchDuration
# Main entry point for Spark Streaming functionality. A StreamingContext represents the
# connection to a Spark cluster, and can be used to create DStream various input sources.
# A Discretized Stream (DStream), the basic abstraction in Spark Streaming, 
# is a continuous sequence of RDDs (of the same type) representing a continuous stream of data
# DStreams can either be created from live data (such as, data from TCP sockets, Kafka, Flume, etc.) using a 
# StreamingContext or it can be generated by transforming existing DStreams using operations such as map and window.
# https://blog.jetoile.fr/2014/05/rdd-quest-ce-que-cest.html
ssc.checkpoint("checkpoint")
# Sets the context to periodically checkpoint the DStream operations for master fault-tolerance.
# Parameters:	directory – HDFS-compatible directory where the checkpoint data will be reliably stored
ssc.remember(1)
# Set each DStreams in this context to remember RDDs it generated in the last given duration. 

In [4]:
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {'volatility': 1})
# createStream(ssc, zkQuorum, groupId, topics)
# Parameters:	
#				ssc – StreamingContext object
#				zkQuorum – Zookeeper quorum (hostname:port,hostname:port,..).
#				groupId – The group id for this consumer.
#				topics – Dict of (topic_name -> numPartitions) to consume. Each partition is consumed in its own thread.
# Topic partitions in Kafka does not correlate to partitions of RDDs generated in Spark Streaming. 

In [5]:
def readInput(line):
    # Convert the input string into a pair of numbers
    print('now')
    vec= [float(x) for x in line.split()] 


    # The pair represents the inputs for two model (forgetting factor 1 and 0.99)
    return [('mod1',('mod1',np.array(vec))),('mod2',('mod2',np.array(vec)))]

In [6]:
def updateFunction(new_values, state): 
    ## RLS update function, formulas slide 51
    ## new_values = (key, [i, y, proxies])
    ## state = what is given by sc.parallelize in next cell on the first call
    ##       = what this function returned on the other calls
    if (len(new_values)>0 ):
        
        key=new_values[0][0]
        yx=new_values[0][1]
        i=yx[0]
        y=yx[1]
        x=yx[2:]
        n=len(x)
        
        beta=state[1]
        beta.shape=(n,1) # Transorms beta in a matrix of n rows and 1 column
        V=state[2]
        mu=state[3]
        sse=state[4]  ## sum of squared errors
        N=state[5]    ## number of treated samples
        x.shape=(1,n)
        err=y-x.dot(beta)        
        sse=sse+pow(err,2.0)
        V=1.0/mu*(V-V.dot(x.T).dot(x).dot(V)/(1.0+float(x.dot(V).dot(x.T)))) # dot = matrix multiplication
                                                                             # .T = Transpose
        gamma=V.dot(x.T)
        beta=beta+gamma*err
        proxyPrediction=x.dot(beta)
        
        errNaivePrec=y-x[0][-1]
        errNaiveMean=y-np.mean(x)
        sseNaivePrec=state[7]
        sseNaiveMean=state[8]
        sseNaivePrec=sseNaivePrec+pow(errNaivePrec,2.0)
        sseNaiveMean=sseNaiveMean+pow(errNaiveMean,2.0)
        
        MSE_RLS=sse/(N+1.0)
        MSE_NaivePrec=sseNaivePrec/(N+1.0)
        MSE_NaiveMean=sseNaiveMean/(N+1.0)
        
        if (key=='mod1'):
            return (key,beta,V,mu,MSE_RLS,N+1, proxyPrediction, sseNaivePrec, sseNaiveMean, MSE_RLS/MSE_NaivePrec, MSE_RLS/MSE_NaiveMean)  ## update formula mod1
        else:
            return (key,beta,V,mu,MSE_RLS,N+1, proxyPrediction, sseNaivePrec, sseNaiveMean, MSE_RLS/MSE_NaivePrec, MSE_RLS/MSE_NaiveMean)  ## update formula mod1
        
    else:
        return state

In [7]:
import re, ast
n=3 # number of features counting t_0
beta1=np.zeros(n)  ## initial parameter vector slide 49
beta2=np.zeros(n)
V1=np.diag(np.zeros(n)+10) ## initial covariance matrix slide 49
V2=np.diag(np.zeros(n)+1)
mu1=1.0 # forgetting factor slide 50
mu2=0.99
data = kvs.map(lambda x: np.array(ast.literal_eval(x[1])))
# map: Returns a new DStream by applying a function to each element of DStream.
# literal_eval: This can be used for safely evaluating strings containing Python values (strings, numbers, tuples, lists, dicts, booleans, and None)
# from untrusted sources without the need to parse the values oneself.
data=data.flatMap(lambda x: [('mod1',('mod1',1.0*np.array(x))),
                            ('mod2',('mod2',1.0*np.array(x)))])
data.pprint() # prints received array
# Return a new DStream by applying a function to all elements of this DStream, and then flattening the results
#initialStateRDD = sc.parallelize([('k',([1,2,3]))])
initialStateRDD = sc.parallelize([(u'mod1', ('mod1',beta1,V1,mu1,0,0,0,0,0)),
                                  (u'mod2', ('mod2',beta2,V2,mu2,0,0,0,0,0))])
# The elements of the collection [] are copied to form a distributed dataset that can be operated on in parallel.
# ('mod1',beta1,V1,mu1,0,0,0)) == what updateFunction returns
data2=data.updateStateByKey(updateFunction,initialRDD=initialStateRDD)
# Return a new "state" DStream where the state for each key is updated by applying the given function on the previous
# state of the key and the new values for the key. This can be used to maintain arbitrary state data for each key.
#data.pprint()
data2.map(lambda x: [x[1][i] for i in [0,1,6,4,9,10]]).pprint() 
# prints key, beta, proxyPrediction, MSE_RLS, NMSE_prec, NMSE_mean

In [8]:
ssc.start()


-------------------------------------------
Time: 2018-03-19 23:14:20
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:14:25
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:14:30
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:14:35
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:14:40
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:14:45
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:14:50
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:14:55
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:15:00
----------

-------------------------------------------
Time: 2018-03-19 23:15:55
-------------------------------------------
['mod2', array([[ 0.03727174],
       [ 0.00200282],
       [ 0.00225778]]), array([[ 0.0374587]]), array([[  3.96670528e-05]]), array([[ 1.38493466]]), array([[ 0.00038658]])]
['mod1', array([[ 0.04322926],
       [ 0.00476823],
       [ 0.00648031]]), array([[ 0.0437197]]), array([[  5.56543238e-05]]), array([[ 1.94311391]]), array([[ 0.00054239]])]

-------------------------------------------
Time: 2018-03-19 23:16:00
-------------------------------------------
('mod1', ('mod1', array([  1.70000000e+01,   1.48668642e-02,   1.00000000e+00,
         1.37664041e-02,   1.52490315e-02])))
('mod2', ('mod2', array([  1.70000000e+01,   1.48668642e-02,   1.00000000e+00,
         1.37664041e-02,   1.52490315e-02])))

-------------------------------------------
Time: 2018-03-19 23:16:00
-------------------------------------------
['mod2', array([[ 0.03389942],
       [ 0.00250634],

-------------------------------------------
Time: 2018-03-19 23:16:40
-------------------------------------------
['mod2', array([[ 0.03802636],
       [ 0.004219  ],
       [ 0.00490579]]), array([[ 0.0386954]]), array([[ 0.00011031]]), array([[ 3.88820171]]), array([[ 0.00107822]])]
['mod1', array([[ 0.03911149],
       [ 0.02525122],
       [ 0.03031116]]), array([[ 0.04318894]]), array([[  8.20508285e-05]]), array([[ 2.89204119]]), array([[ 0.00080198]])]

-------------------------------------------
Time: 2018-03-19 23:16:45
-------------------------------------------
('mod1', ('mod1', array([  3.40000000e+01,   3.31197425e-02,   1.00000000e+00,
         4.18008170e-02,   3.29977223e-02])))
('mod2', ('mod2', array([  3.40000000e+01,   3.31197425e-02,   1.00000000e+00,
         4.18008170e-02,   3.29977223e-02])))
('mod1', ('mod1', array([  3.50000000e+01,   2.51212886e-02,   1.00000000e+00,
         3.29977223e-02,   3.31197425e-02])))
('mod2', ('mod2', array([  3.50000000e+01,   2

-------------------------------------------
Time: 2018-03-19 23:17:25
-------------------------------------------
('mod1', ('mod1', array([  4.70000000e+01,   3.56078093e-02,   1.00000000e+00,
         3.35345087e-02,   3.90186310e-02])))
('mod2', ('mod2', array([  4.70000000e+01,   3.56078093e-02,   1.00000000e+00,
         3.35345087e-02,   3.90186310e-02])))
('mod1', ('mod1', array([  4.80000000e+01,   3.30287074e-02,   1.00000000e+00,
         3.90186310e-02,   3.56078093e-02])))
('mod2', ('mod2', array([  4.80000000e+01,   3.30287074e-02,   1.00000000e+00,
         3.90186310e-02,   3.56078093e-02])))
('mod1', ('mod1', array([  4.90000000e+01,   3.04481926e-02,   1.00000000e+00,
         3.56078093e-02,   3.30287074e-02])))
('mod2', ('mod2', array([  4.90000000e+01,   3.04481926e-02,   1.00000000e+00,
         3.56078093e-02,   3.30287074e-02])))
('mod1', ('mod1', array([  5.00000000e+01,   2.78662643e-02,   1.00000000e+00,
         3.30287074e-02,   3.04481926e-02])))
('mod2', ('

-------------------------------------------
Time: 2018-03-19 23:17:55
-------------------------------------------
['mod2', array([[ 0.03107649],
       [ 0.0055396 ],
       [ 0.00673577]]), array([[ 0.03148659]]), array([[  5.06187027e-07]]), array([[ 0.02494043]]), array([[  4.84801295e-06]])]
['mod1', array([[ 0.03010417],
       [ 0.03868449],
       [ 0.04871366]]), array([[ 0.03302332]]), array([[  1.39391040e-07]]), array([[ 0.00686796]]), array([[  1.33501953e-06]])]

-------------------------------------------
Time: 2018-03-19 23:18:00
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 23:18:00
-------------------------------------------
['mod2', array([[ 0.03107649],
       [ 0.0055396 ],
       [ 0.00673577]]), array([[ 0.03148659]]), array([[  5.06187027e-07]]), array([[ 0.02494043]]), array([[  4.84801295e-06]])]
['mod1', array([[ 0.03010417],
       [ 0.03868449],
       [ 0.04871366]]), array([[ 0.03302332]]), array(

...

-------------------------------------------
Time: 2018-03-19 23:41:10
-------------------------------------------
('mod1', ('mod1', array([  6.18000000e+02,   1.59807654e-02,   1.00000000e+00,
         1.31285980e-02,   1.61139149e-02])))
('mod2', ('mod2', array([  6.18000000e+02,   1.59807654e-02,   1.00000000e+00,
         1.31285980e-02,   1.61139149e-02])))
('mod1', ('mod1', array([  6.19000000e+02,   1.81420553e-02,   1.00000000e+00,
         1.61139149e-02,   1.59807654e-02])))
('mod2', ('mod2', array([  6.19000000e+02,   1.81420553e-02,   1.00000000e+00,
         1.61139149e-02,   1.59807654e-02])))
('mod1', ('mod1', array([  6.20000000e+02,   2.03026599e-02,   1.00000000e+00,
         1.59807654e-02,   1.81420553e-02])))
('mod2', ('mod2', array([  6.20000000e+02,   2.03026599e-02,   1.00000000e+00,
         1.59807654e-02,   1.81420553e-02])))
('mod1', ('mod1', array([  6.21000000e+02,   2.24625797e-02,   1.00000000e+00,
         1.81420553e-02,   2.03026599e-02])))
('mod2', ('

-------------------------------------------
Time: 2018-03-19 23:41:45
-------------------------------------------
('mod1', ('mod1', array([  6.35000000e+02,   1.56998994e-02,   1.00000000e+00,
         1.17238602e-02,   1.25658461e-02])))
('mod2', ('mod2', array([  6.35000000e+02,   1.56998994e-02,   1.00000000e+00,
         1.17238602e-02,   1.25658461e-02])))

-------------------------------------------
Time: 2018-03-19 23:41:45
-------------------------------------------
['mod2', array([[ 0.01118676],
       [ 0.16679233],
       [ 0.19108608]]), array([[ 0.01554337]]), array([[  1.13833561e-09]]), array([[  5.86318571e-05]]), array([[  1.06148188e-08]])]
['mod1', array([[ 0.00751113],
       [ 0.26446825],
       [ 0.32412744]]), array([[ 0.01468465]]), array([[  5.82743425e-09]]), array([[ 0.00030015]]), array([[  5.43400011e-08]])]

-------------------------------------------
Time: 2018-03-19 23:41:50
-------------------------------------------

----------------------------------

-------------------------------------------
Time: 2018-03-19 23:42:25
-------------------------------------------
['mod2', array([[ 0.01073718],
       [ 0.16877201],
       [ 0.19334991]]), array([[ 0.0108973]]), array([[  5.50817986e-07]]), array([[ 0.02892191]]), array([[  5.13324319e-06]])]
['mod1', array([[ 0.00734201],
       [ 0.26560259],
       [ 0.32550562]]), array([[ 0.00760242]]), array([[  2.57834523e-07]]), array([[ 0.01353817]]), array([[  2.40283967e-06]])]

-------------------------------------------
Time: 2018-03-19 23:42:30
-------------------------------------------
('mod1', ('mod1', array([  6.53000000e+02,   5.43170051e-06,   1.00000000e+00,
         6.77305177e-06,   6.77814378e-06])))
('mod2', ('mod2', array([  6.53000000e+02,   5.43170051e-06,   1.00000000e+00,
         6.77305177e-06,   6.77814378e-06])))

-------------------------------------------
Time: 2018-03-19 23:42:30
-------------------------------------------
['mod2', array([[ 0.01057945],
       [ 0

-------------------------------------------
Time: 2018-03-19 23:43:10
-------------------------------------------
('mod1', ('mod1', array([  6.66000000e+02,   4.51977672e-03,   1.00000000e+00,
         6.76116336e-03,   5.64367265e-03])))
('mod2', ('mod2', array([  6.66000000e+02,   4.51977672e-03,   1.00000000e+00,
         6.76116336e-03,   5.64367265e-03])))
('mod1', ('mod1', array([  6.67000000e+02,   6.62224662e-03,   1.00000000e+00,
         5.64367265e-03,   4.51977672e-03])))
('mod2', ('mod2', array([  6.67000000e+02,   6.62224662e-03,   1.00000000e+00,
         5.64367265e-03,   4.51977672e-03])))
('mod1', ('mod1', array([  6.68000000e+02,   8.72611394e-03,   1.00000000e+00,
         4.51977672e-03,   6.62224662e-03])))
('mod2', ('mod2', array([  6.68000000e+02,   8.72611394e-03,   1.00000000e+00,
         4.51977672e-03,   6.62224662e-03])))
('mod1', ('mod1', array([  6.69000000e+02,   1.08313792e-02,   1.00000000e+00,
         6.62224662e-03,   8.72611394e-03])))
('mod2', ('

-------------------------------------------
Time: 2018-03-19 23:43:45
-------------------------------------------
('mod1', ('mod1', array([  6.83000000e+02,   4.71851122e-02,   1.00000000e+00,
         2.83512139e-02,   3.77754346e-02])))
('mod2', ('mod2', array([  6.83000000e+02,   4.71851122e-02,   1.00000000e+00,
         2.83512139e-02,   3.77754346e-02])))

-------------------------------------------
Time: 2018-03-19 23:43:45
-------------------------------------------
['mod2', array([[ 0.01027885],
       [ 0.17307935],
       [ 0.19992844]]), array([[ 0.02273824]]), array([[  2.78497948e-06]]), array([[ 0.1486392]]), array([[  2.59466455e-05]])]
['mod1', array([[ 0.00714835],
       [ 0.26751691],
       [ 0.32937911]]), array([[ 0.02717522]]), array([[  1.83519664e-06]]), array([[ 0.09794764]]), array([[  1.70978627e-05]])]



In [9]:
ssc.stop(stopSparkContext=False,stopGraceFully=True)

-------------------------------------------
Time: 2018-03-19 17:54:25
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 17:54:25
-------------------------------------------
['mod2', array([[  1.14151631e-08],
       [  2.17159392e-15],
       [  2.18195076e-15]]), array([[  1.14151631e-08]]), array([[  1.30719998e-17]]), array([[ 1.02368855]]), array([[  1.17648000e-16]])]
['mod1', array([[  1.26100554e-08],
       [  1.86921790e-14],
       [  1.90980643e-14]]), array([[  1.26100554e-08]]), array([[  1.59337649e-17]]), array([[ 1.2477978]]), array([[  1.43403885e-16]])]

-------------------------------------------
Time: 2018-03-19 17:54:30
-------------------------------------------

-------------------------------------------
Time: 2018-03-19 17:54:30
-------------------------------------------
['mod2', array([[  1.14151631e-08],
       [  2.17159392e-15],
       [  2.18195076e-15]]), array([[  1.14151631e-08]]), array([[  1.307

In [ ]:
a=[1,2,3,4]


In [ ]:
[a[i] for i in [1,0,2,3]]
